In [55]:
import plotly.express as px
import csv
import numpy as np
lvl = {
    'CameraRGBDSensor::StartPublishDepth':1,
    'CameraRGBDSensor::EndPublishDepth':1,
    'CameraRGBDSensor::StartPublishRGB':1,
    'CameraRGBDSensor::EndPublishRGB':1,
    'CameraSensor::RequestFrame':0,
}
ignore_events={'CameraSensor::SetupPasses','CameraRGBDSensor::StartPublishRGB','CameraRGBDSensor::StartPublishDepth' }
DATASET="/home/michal/Desktop/TestRGBD/CSVs/RGB/"
cameras = []
timestamps = []
events = []
y = []
input_files =[]

input_files.append(DATASET+'/events_FRONT_FRONT.csv')
input_files.append(DATASET+'/events_BACK_PORTBOARD.csv')
input_files.append(DATASET+'/events_PORTBOARD_PORTBOARD.csv')
input_files.append(DATASET+'/events_STARBOARD_STARBOARD.csv')

for camera_id, inputFile in enumerate(input_files) :
    with open(inputFile, 'r') as file:
        csvreader = csv.reader(file, delimiter='\t')
        header = next(csvreader)
        for row in csvreader:
            if row[1] not in ignore_events:
                timestamps.append(float(row[0])/1000000)
                events.append(row[1])
                cameras.append(camera_id)        
for camera_id, event in zip(cameras,events) :
    if event in lvl:
        y.append(lvl[event]*0.1+camera_id)
    else:
        y.append(0)

In [56]:
fig = px.scatter( x=timestamps, y=y, symbol=cameras, color=events,  labels={
                     "x":"Execution Time(s)",
                     "y":"Camera Number",
                     "color" : "Event",
                     "symbol" : "Camera Number",
    
                 },)
fig.show()


In [53]:
durations={}
last_ts = {}

for camera_id in cameras:
    last_ts[camera_id] = 0
    durations[camera_id] = []
    
for camera_id, event, timestamp in zip(cameras,events, timestamps):
    if (event == 'CameraSensor::RequestFrame'):
        if last_ts[camera_id] != 0:
            delta = timestamp - last_ts[camera_id]
            durations[camera_id].append(delta)
        last_ts[camera_id] = timestamp

In [54]:
for camera_id in last_ts.keys():
    print ("camera %d, avg : %f ms std_dev: %f ms" %(camera_id, 
                                               np.average(durations[camera_id]), 
                                               np.std(durations[camera_id])))

camera 0, avg : 0.046309 ms std_dev: 0.018214 ms
